In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

In [2]:
from quantopian.pipeline.filters import Q1500US

In [3]:
universe = Q1500US()

In [4]:
from quantopian.pipeline.data import morningstar

In [5]:
sector = morningstar.asset_classification.morningstar_sector_code.latest

In [7]:
energy_sector = sector.eq(309) #eq means equal sector code

In [8]:
#Masking to use before/combine with filters
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

In [9]:
dollar_volume = AverageDollarVolume(window_length=30)

In [12]:
high_dollar_volume = dollar_volume.percentile_between(90,100)

In [13]:
top_open_prices = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)

In [ ]:
# STEPS FOR PIPELINE:

In [15]:
def make_pipeline():
    # Universe Q1500US
    base_universe = Q1500US()

    # Energy Sector
    energy_sector = sector.eq(309)

    # Make Mask of 1500US and Energy
    base_energy = base_universe & energy_sector

    # Dollar Volume (30 days) Grab the Info
    dollar_volume = AverageDollarVolume(window_length=30)

    # Grab the top 5% in avg dollar volume
    high_dollar_volume = dollar_volume.percentile_between(95,100)

    # Combine the filters
    top_five_base_energy = base_energy & high_dollar_volume

    # 10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=top_five_base_energy)

    # 30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=top_five_base_energy)

    # Percent difference
    percent_difference = (mean_10 - mean_30) / mean_10          

    # List of Shorts
    shorts = percent_difference < 0

    # List of Longs
    longs = percent_difference > 0

    # Final Mask/Filter for anything in shorts or long
    stocks_to_trade = (shorts | longs)

    # Return Pipeline
    return Pipeline(columns={
        'longs': longs,
        'shorts': shorts,
        'perc_diff': percent_difference
    }, screen=stocks_to_trade)

In [16]:
result = run_pipeline(make_pipeline(),'2015-05-05','2015-05-05')
result

longs  perc_diff shorts
2015-05-05 00:00:00+00:00 Equity(216 [HES])     True   0.035240  False
                          Equity(448 [APA])     True   0.034681  False
                          Equity(455 [APC])     True   0.046850  False
                          Equity(858 [BHI])     True   0.032702  False
                          Equity(1746 [COG])    True   0.054890  False
                          Equity(2368 [DVN])    True   0.044218  False
                          Equity(2564 [EOG])    True   0.031103  False
                          Equity(2621 [VAL])    True   0.056779  False
                          Equity(3443 [HAL])    True   0.046945  False
                          Equity(3647 [HP])     True   0.039377  False
                          Equity(5035 [MRO])    True   0.058024  False
                          Equity(5213 [NBL])    True   0.010335  False
                          Equity(5214 [NBR])    True   0.060268  False
                          Equity(5249 [NE])     True   0.036200  False
                          Equity(5729 [OXY])    True   0.028915  False
                          Equity(6928 [SLB])    True   0.044484  False
                          Equity(7244 [SWN])    True   0.066108  False
                          Equity(7612 [ANDV])   True   0.005961  False
                          Equity(7990 [VLO])   False  -0.017444   True
                          Equity(8214 [WMB])    True   0.018526  False
                          Equity(8347 [XOM])    True   0.017047  False
                          Equity(8461 [CHK])    True   0.014064  False
                          Equity(9038 [RIG])    True   0.045965  False
                          Equity(13176 [CAM])   True   0.075880  False
                          Equity(17436 [PXD])   True   0.010144  False
                          Equity(19249 [RRC])   True   0.080089  False
                          Equity(19336 [WFT])   True   0.046839  False
                          Equity(22784 [FTI])   True   0.051709  False
                          Equity(23112 [CVX])   True   0.018619  False
                          Equity(23998 [COP])   True   0.023344  False
                          Equity(24809 [NOV])   True   0.024333  False
                          Equity(25707 [WLL])   True   0.045988  False
                          Equity(33856 [CLR])   True   0.060419  False
                          Equity(34440 [CXO])   True   0.040477  False
                          Equity(39797 [OAS])   True   0.040664  False
                          Equity(40852 [KMI])   True   0.022498  False
                          Equity(41636 [MPC])   True   0.011811  False
                          Equity(42788 [PSX])   True   0.020482  False